In [1]:
!pip install transformers datasets
!pip install conllu
!pip3 install torchvision
!pip install scikit-learn

In [2]:
import sys
sys.path.append('../API')

import api

import torch
import torch.nn as nn

import numpy

from transformers import AutoTokenizer, AutoModel

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


C:\Users\dylanw2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Connect to DB with your credentials
# Syntax: db = api.connect_to_db("username","password")
db = api.connect_to_db()

Pinged your deployment. You successfully connected to MongoDB!


In [11]:
# API call to get training and testing data
# 80/20 train/test split with randomizing to avoid biased training
training_data, testing_data = api.get_database_content(db, .8, .2, 1)

# Training Data Labels
training_features = []
training_true_labels = []
for item in training_data:
    training_features.append(item['title'])
    training_true_labels.append(item['label'])

# Testing Data Labels
testing_features = []
testing_true_labels = []
for item in testing_data:
    testing_features.append(item['title'])
    testing_true_labels.append(item['label'])

print(training_features)
print(training_true_labels)
print(testing_features)
print(testing_true_labels)

['US pledges to “continue to support Israel” as over 1,000 more Gazans are massacred over the weekend', 'Pro-Israel man commits hate crime against peaceful protesters in San Jose, CA', 'Zionism: The Secret Evidence Israel Tried to Bury', 'Why are so many of the hostages elderly?', 'Jeremy Scahill of The Intercept destroys Israeli propaganda around Al-Shifa hospital', 'How do you deal with socialist and leftist friends who hate Israel?', 'Ten Israeli soldiers, including senior colonel, killed in Gaza ambush', 'Letter from released hostages to the leaders and fighters of Al Qassam', 'Why Netanyahu Bolstered Hamas — The Israeli prime minister followed a decades-old divide-and-rule strategy that fuels endless war.', 'Why Gaza is not Singapore', 'Dear Israel, please pay me for my “Propaganda”', 'UN General Assembly votes overwhelmingly in favour of Gaza ceasefire', 'A second Michigan Senate candidate says he was offered $20M to run against Rashida Tlaib', 'A recommended reading list about P

In [13]:
# Convert the -1, 0, 1 labels to 0, 1, 2 for Pytorch

def map_labels(labels):
    return [label + 1 for label in labels]

training_true_labels = map_labels(training_true_labels)
testing_true_labels = map_labels(testing_true_labels)

print(training_features)
print(training_true_labels)
print(testing_features)
print(testing_true_labels)

['US pledges to “continue to support Israel” as over 1,000 more Gazans are massacred over the weekend', 'Pro-Israel man commits hate crime against peaceful protesters in San Jose, CA', 'Zionism: The Secret Evidence Israel Tried to Bury', 'Why are so many of the hostages elderly?', 'Jeremy Scahill of The Intercept destroys Israeli propaganda around Al-Shifa hospital', 'How do you deal with socialist and leftist friends who hate Israel?', 'Ten Israeli soldiers, including senior colonel, killed in Gaza ambush', 'Letter from released hostages to the leaders and fighters of Al Qassam', 'Why Netanyahu Bolstered Hamas — The Israeli prime minister followed a decades-old divide-and-rule strategy that fuels endless war.', 'Why Gaza is not Singapore', 'Dear Israel, please pay me for my “Propaganda”', 'UN General Assembly votes overwhelmingly in favour of Gaza ceasefire', 'A second Michigan Senate candidate says he was offered $20M to run against Rashida Tlaib', 'A recommended reading list about P

In [14]:
class MyClassifier(nn.Module):
    def __init__(self, num_labels=3):
        super(MyClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("bert-base-cased")
        self.fc = nn.Linear(self.bert.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        return self.fc(pooled_output)


In [15]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
vocab_size = len(tokenizer)

def tokenize_rte(text_data, labels, tokenizer):
    tokenized_data = []

    for text, label in zip(text_data, labels):
        encoded_input = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
        tokenized_instance = {
            'input_ids': encoded_input['input_ids'][0],
            'attention_mask': encoded_input['attention_mask'][0],
            'label': label,
            'original_text': text
        }
        tokenized_data.append(tokenized_instance)

    return tokenized_data

tokenized_training_data = tokenize_rte(training_features, training_true_labels, tokenizer)
print(tokenized_training_data)


[{'input_ids': tensor([  101,  1646, 20335,  1116,  1106,   789,  2760,  1106,  1619,  3103,
          790,  1112,  1166,   122,   117,  1288,  1167, 15109,  2316,  1132,
        11584,  1181,  1166,  1103,  5138,   102,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,

In [18]:
# Define training function
def train(model, data_inst, data_labels, weight_adjuster, loss_fn,
          batch_size=100, num_epochs=5, epoch_callback=None):
    data_labels = torch.tensor(data_labels, dtype=torch.long)

    for epoch in range(num_epochs):
        print("Epoch %d" % epoch)
        num_batches = int(torch.ceil(torch.tensor(len(data_inst) / batch_size)))

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, len(data_inst))

            # Extract batch data
            batch = data_inst[batch_start:batch_end]
            input_ids = torch.stack([b['input_ids'] for b in batch])
            attention_mask = torch.stack([b['attention_mask'] for b in batch])
            batch_labels = data_labels[batch_start:batch_end]

            weight_adjuster.zero_grad()

            # Forward pass
            logits = model(input_ids, attention_mask)

            loss = loss_fn(logits, batch_labels)
            print(f"Epoch {epoch}, Batch {batch_idx}: {batch_start} --> {batch_end}, Batch loss {loss.item()}")

            loss.backward()
            weight_adjuster.step()

        if epoch_callback is not None:
            epoch_callback()

    return model



model = MyClassifier()
loss_fn = torch.nn.CrossEntropyLoss()
weight_adjuster = torch.optim.Adam(model.parameters())
# Run the training
train(model=model,
      data_inst=tokenized_training_data,
      data_labels=training_true_labels,
      weight_adjuster=weight_adjuster,
      loss_fn=loss_fn,
      batch_size=1,
      num_epochs=1)


Epoch 0
Epoch 0, Batch 0: 0 --> 1, Batch loss 1.2243051528930664
Epoch 0, Batch 1: 1 --> 2, Batch loss 0.5608019232749939
Epoch 0, Batch 2: 2 --> 3, Batch loss 0.04316011071205139
Epoch 0, Batch 3: 3 --> 4, Batch loss 4.537389755249023
Epoch 0, Batch 4: 4 --> 5, Batch loss 4.137518882751465
Epoch 0, Batch 5: 5 --> 6, Batch loss 3.177686929702759
Epoch 0, Batch 6: 6 --> 7, Batch loss 2.503993511199951
Epoch 0, Batch 7: 7 --> 8, Batch loss 1.7405083179473877
Epoch 0, Batch 8: 8 --> 9, Batch loss 1.3996661901474
Epoch 0, Batch 9: 9 --> 10, Batch loss 2.103330612182617
Epoch 0, Batch 10: 10 --> 11, Batch loss 0.9396719932556152
Epoch 0, Batch 11: 11 --> 12, Batch loss 1.2065856456756592
Epoch 0, Batch 12: 12 --> 13, Batch loss 0.572638988494873
Epoch 0, Batch 13: 13 --> 14, Batch loss 0.46923527121543884
Epoch 0, Batch 14: 14 --> 15, Batch loss 3.8102540969848633
Epoch 0, Batch 15: 15 --> 16, Batch loss 1.4619841575622559
Epoch 0, Batch 16: 16 --> 17, Batch loss 1.3076763153076172
Epoch 0,

MyClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affi

In [19]:

def evaluate(model, data_inst, data_labels, batch_size=100):
    model.eval()  # Set the model to evaluation mode

    predictions = []
    actuals = []

    with torch.no_grad():  # No need to track gradients during evaluation
        num_batches = int(torch.ceil(torch.tensor(len(data_inst) / batch_size)))

        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, len(data_inst))

            batch = data_inst[batch_start:batch_end]
            input_ids = torch.stack([b['input_ids'] for b in batch])
            attention_mask = torch.stack([b['attention_mask'] for b in batch])
            labels = torch.tensor(data_labels[batch_start:batch_end], dtype=torch.long)

            logits = model(input_ids, attention_mask)
            preds = torch.argmax(logits, dim=1).tolist()  # Convert logits to predicted class indices

            predictions.extend(preds)
            actuals.extend(labels.tolist())

    # Calculate evaluation metrics
    accuracy = accuracy_score(actuals, predictions)
    precision = precision_score(actuals, predictions, average='macro')
    recall = recall_score(actuals, predictions, average='macro')
    f1 = f1_score(actuals, predictions, average='macro')

    return accuracy, precision, recall, f1


tokenized_testing_data = tokenize_rte(testing_features, testing_true_labels, tokenizer)
# Example usage
accuracy, precision, recall, f1 = evaluate(model, tokenized_testing_data, testing_true_labels, batch_size=10)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")

Accuracy: 0.5084745762711864, Precision: 0.1694915254237288, Recall: 0.3333333333333333, F1 Score: 0.2247191011235955


C:\Users\dylanw2\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
